In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# $\mathbb{S}^2$ Sphere Geometry

In [ ]:
from src.manifolds.S2 import *
M = S2()
print(M)

from src.plotting import *
M.plot()

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)

# element, tangent vector and covector
x = np.array([0.,0.])
v = np.array([-1.,1.])
p = M.flatf(x,v)

print("x = ", x)
print("v = ", v)
print("p = ", p)

## Riemannian Geodesics

In [ ]:
# 2nd order geodesic equation
from src.Riemannian import geodesic
geodesic.initialize(M)
xs = M.Exptf(x,v)

%matplotlib notebook
newfig()
M.plot(rotate=(30,-45))
M.plotx(xs,v,linewidth = 1.5, s=50)
plt.show()

### Geodesics from Hamiltonian equations

In [ ]:
# Hamiltonian dynamics
q = x
print(M.Hf(q,p))
from src.dynamics import Hamiltonian
Hamiltonian.initialize(M)

# geodesic
qs = M.Exp_Hamiltoniantf(q,p).T
newfig()
%matplotlib inline
M.plot(rotate=(30,-45))
M.plotx(qs,v,linewidth=1.5,s=50)
plt.show()
#plt.savefig("geodesicOnSphere.pdf")
(ts,qps) = M.Hamiltonian_dynamicsf(q,p)
ps = qps[:,1,:]
print("Energy: ",np.array([M.Hf(q,p) for (q,p) in zip(qs,ps)]))

# plot geodesic on R^2
%matplotlib inline
plt.figure()
M.plotR2x(qs,v)
plt.show()

## Curvature

In [ ]:
from src.Riemannian import curvature
curvature.initialize(M)
print("curvature = ", M.Rf(x))

## Parallel Transport

In [ ]:
# Parallel transport
from src.Riemannian import parallel_transport
parallel_transport.initialize(M)

v = np.array([-1./2,-1./2])
v = v/M.normf(x,v)
t = np.linspace(0,1,n_steps.get_value())
gamma = np.vstack([t**2,-np.sin(t)]).T
dgamma = np.vstack([2*t,np.cos(t)]).T

# test parallel transport
vt = M.parallel_transportf(v,gamma,dgamma)
# plot of result
M.plot(rotate=(30,-45))
M.plotx(gamma,v=vt)
plt.show()
#plt.savefig("partrans.pdf")

## Brownian Motion

In [ ]:
# coordinate form
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

(ts,xs) = M.Brownian_coordsf(x,dWsf(M.dim.eval()))
newfig()
M.plot()
M.plotx(xs)
plt.show()

In [ ]:
# Brownian Motion from stochastic development
from src.stochastic import Stochastic_Development
from src.stochastic import Brownian_Stochastic_Development
Brownian_Stochastic_Development.initialize(M)
# test Simulation of Brownian Motion
qsv = SD_brownian(q0)
newfig()
plotM()
plotFMx(qsv,)
plt.show()